In [1]:
import pandas as pd
import numpy as np

from gscore.osw.peakgroups import fetch_peak_groups
from gscore.osw.queries import (
    FETCH_EXPORT_DATA
)
from gscore.models.denoiser import DenoizingClassifier
from gscore.osw.connection import create_table

from gscore.models.preprocess import STANDARD_SCALAR_PIPELINE
from gscore.models.distributions import build_false_target_protein_distributions, ScoreDistribution

In [3]:
osw_path = '/home/aaron/projects/gscorer/data/openswath/temp/Simon_S1608_052.osw'

In [4]:

peak_groups = fetch_peak_groups(
    host=osw_path, 
    query=FETCH_EXPORT_DATA
)

# peak_groups.rerank_groups(
#     rerank_keys=['var_xcorr_shape'], 
#     ascending=False
# )

# highest_ranking = peak_groups.select_peak_group(
#     rank=1,
#     rerank_keys=['var_xcorr_shape'], 
#     ascending=False
# )
    

In [5]:
all_peak_groups = peak_groups.select_peak_group(
    return_all=True
)

In [5]:
# highest_ranking_swath_data = all_peak_groups.loc[
#     all_peak_groups.groupby(['transition_group_id'])['var_xcorr_shape_weighted'].idxmax()
# ]

In [7]:
cutoff = 0.01

In [8]:
pass_threshold = all_peak_groups.loc[
    all_peak_groups.groupby(['transition_group_id'])['m_score'].idxmin()
]

# Need to fix this if both have 0.0 q value to take the highest scoring

In [13]:
pass_threshold = pass_threshold[(pass_threshold['m_score'] <= cutoff)].copy()

In [14]:
print(len(pass_threshold[pass_threshold['target'] == 0.0]), len(pass_threshold[pass_threshold['target'] == 1.0]))

0 11634


In [15]:
pass_threshold.columns

Index(['transition_group_id', 'feature_id', 'exp_rt', 'norm_rt', 'delta_rt',
       'mz', 'charge', 'decoy', 'peptide_sequence',
       'modified_peptide_sequence', 'protein_accession', 'protein_decoy',
       'area_intensity', 'apex_intensity', 'ms2_area_intensity',
       'total_area_intensity', 'vote_percentage', 'd_score', 'm_score',
       'alt_d_score', 'ghost_score_id', 'target'],
      dtype='object')

In [16]:
pass_threshold['peptide_sequence_charge'] = pass_threshold.apply(
    lambda row: '{}_{}'.format(row['peptide_sequence'], row['charge']),
    axis=1
)

In [17]:
proteotypic_counts = pd.DataFrame(
    pass_threshold['peptide_sequence_charge'].value_counts(),
).reset_index()

In [18]:
proteotypic_counts.columns = ['peptide_charge', 'count']

In [19]:
proteotypic_peptides = list(
    proteotypic_counts[
        proteotypic_counts['count'] == 1
    ]['peptide_charge']
)

In [20]:
len(proteotypic_peptides)

10845

In [21]:
len(pass_threshold[pass_threshold['target'] == 1.0].groupby(['protein_accession']))

5097

In [22]:
protein_groups = pass_threshold[pass_threshold['target'] == 1.0].groupby(['protein_accession'])

In [23]:
pg_df = [group for _, group in protein_groups]

In [24]:
protein_groups = list()
for group in pg_df:
    if len(group) >= 2:
        protein_groups.append(group)

In [25]:
len(protein_groups)

2370